# HW07 Векторные представления

## 1 Векторизуем тексты

In [6]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
from razdel import tokenize as razdel_tokenize
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    
    tokens = [token.text for token in list(razdel_tokenize(text))]
    tokens = [token for token in tokens if token.isalnum()]

    return ' '.join(tokens)

In [10]:
corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [11]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [50]:
data.head()

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилище решить ...,правило внесудебный проникновение полицейский ...
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия


In [30]:
lenta = open('lenta.txt').read().splitlines()[:1000]
data_lenta = [normalize(text) for text in lenta]

In [41]:
print(data_lenta[1])

русский инвалид 16 сентябрь 1914 года.министерство народный просвещение вид происходить чрезвычайный событие признать соответственный день годовщина день рождение м.ю лермонтов 2-й октябрь 1914 год ограничиться совершение учебный заведение панихида поэт отложить празднование юбилей благоприятный время


In [42]:
my_w2v = gensim.models.Word2Vec([text.split() for text in data_lenta], size=50, sg=1)

In [47]:
ready_w2v = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [48]:
def get_embedding(text, model, dim):
    text = text.split()
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total)
        except (KeyError, ValueError):
            continue
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    return vector

In [53]:
len(data.text_1_norm)

7227

In [56]:
dim = 50
X_1_my_w2v = np.zeros((len(data.text_1_norm), dim))
X_2_my_w2v = np.zeros((len(data.text_2_norm), dim))

for i, text in enumerate(data.text_1_norm.values):
    X_1_my_w2v[i] = get_embedding(text, my_w2v, dim)
    
for i, text in enumerate(data.text_2_norm.values):
    X_2_my_w2v[i] = get_embedding(text, my_w2v, dim)

X_my_w2v = np.concatenate([X_1_my_w2v, X_2_my_w2v], axis=1)
y = data['label'].values

<ipython-input-48-34c80056d91d>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  v = model[word]


In [57]:
dim = 50
X_1_ready_w2v = np.zeros((len(data.text_1_norm), dim))
X_2_ready_w2v = np.zeros((len(data.text_2_norm), dim))

for i, text in enumerate(data.text_1_norm.values):
    X_1_ready_w2v[i] = get_embedding(text, ready_w2v, dim)
    
for i, text in enumerate(data.text_2_norm.values):
    X_2_ready_w2v[i] = get_embedding(text, ready_w2v, dim)

X_ready_w2v = np.concatenate([X_1_ready_w2v, X_2_ready_w2v], axis=1)

In [58]:
from sklearn.model_selection import cross_val_score

In [73]:
clf_lr = LogisticRegression(C=10000)

In [74]:
%%capture --no-stdout _
print(cross_val_score(clf_lr, X_my_w2v, y, scoring="f1_micro"))
print(cross_val_score(clf_lr, X_ready_w2v, y, scoring="f1_micro"))

[0.44329184 0.45159059 0.48650519 0.38823529 0.41038062]
[0.40940526 0.40940526 0.40899654 0.40899654 0.40899654]


Обе модели справляются достаточно одинаково

## 2. Преобразуем тексты в векторы в каждой паре 5 методами - SVD, NMF, Word2Vec (свой и русвекторовский), Fastext.

In [75]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data.text_1_norm, data.text_2_norm]))

TfidfVectorizer(max_df=0.4, max_features=1000, min_df=3)

In [78]:
svd = TruncatedSVD(200)

X_1_svd = svd.fit_transform(tfidf.transform(data.text_1_norm))
X_2_svd = svd.fit_transform(tfidf.transform(data.text_2_norm))

X_svd = np.concatenate([X_1_svd, X_2_svd], axis=1)

In [80]:
%%capture --no-stdout _
nmf = NMF(100)

X_1_nmf = nmf.fit_transform(tfidf.transform(data.text_1_norm))
X_2_nmf = nmf.fit_transform(tfidf.transform(data.text_2_norm))

X_nmf = np.concatenate([X_1_nmf, X_2_nmf], axis=1)

In [82]:
fast_text = gensim.models.FastText([text.split() for text in data_lenta], size=50, min_n=4, max_n=8) 

X_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_2_ft[i] = get_embedding(text, fast_text, dim)
    
X_ft = np.concatenate([X_1_ft, X_2_ft], axis=1)

<ipython-input-48-34c80056d91d>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  v = model[word]


In [86]:
from sklearn.metrics.pairwise import cosine_distances

In [87]:
my_w2v_dist = cosine_distances(X_1_my_w2v, X_2_my_w2v)
ready_w2v_dist = cosine_distances(X_1_ready_w2v, X_2_ready_w2v)
svd_dist = cosine_distances(X_1_svd, X_2_svd)
nmf_dist = cosine_distances(X_1_nmf, X_2_nmf)
ft_dist = cosine_distances(X_1_ft, X_2_ft)

In [88]:
distances = np.concatenate((my_w2v_dist, ready_w2v_dist, svd_dist, nmf_dist, ft_dist), axis=1)

In [91]:
clf2_lr = LogisticRegression(C=1000, class_weight='balanced')

In [92]:
%%capture --no-stdout _
print(cross_val_score(clf_lr, distances, y, scoring="f1_micro"))

[0.41839557 0.44190871 0.45190311 0.38961938 0.40968858]


Считался намного дольше, но результат стал не сильно лучше

#### Изменим параметры

In [93]:
tfidf = TfidfVectorizer(min_df=5, max_df=0.9, max_features=1500)
tfidf.fit(pd.concat([data.text_1_norm, data.text_2_norm]))

TfidfVectorizer(max_df=0.9, max_features=1500, min_df=5)

In [94]:
svd = TruncatedSVD(400)

X_1_svd = svd.fit_transform(tfidf.transform(data.text_1_norm))
X_2_svd = svd.fit_transform(tfidf.transform(data.text_2_norm))

X_svd = np.concatenate([X_1_svd, X_2_svd], axis=1)

In [95]:
%%capture --no-stdout _
nmf = NMF(200)

X_1_nmf = nmf.fit_transform(tfidf.transform(data.text_1_norm))
X_2_nmf = nmf.fit_transform(tfidf.transform(data.text_2_norm))

X_nmf = np.concatenate([X_1_nmf, X_2_nmf], axis=1)

In [96]:
fast_text = gensim.models.FastText([text.split() for text in data_lenta], size=50, min_n=4, max_n=8) 

dim = 150

X_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_2_ft[i] = get_embedding(text, fast_text, dim)
    
X_ft = np.concatenate([X_1_ft, X_2_ft], axis=1)

<ipython-input-48-34c80056d91d>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  v = model[word]


In [99]:
dim = 300

X_1_my_w2v = np.zeros((len(data.text_1_norm), dim))
X_2_my_w2v = np.zeros((len(data.text_2_norm), dim))

for i, text in enumerate(data.text_1_norm.values):
    X_1_my_w2v[i] = get_embedding(text, my_w2v, dim)
    
for i, text in enumerate(data.text_2_norm.values):
    X_2_my_w2v[i] = get_embedding(text, my_w2v, dim)

X_my_w2v = np.concatenate([X_1_my_w2v, X_2_my_w2v], axis=1)

<ipython-input-48-34c80056d91d>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  v = model[word]


In [100]:
dim = 400
X_1_ready_w2v = np.zeros((len(data.text_1_norm), dim))
X_2_ready_w2v = np.zeros((len(data.text_2_norm), dim))

for i, text in enumerate(data.text_1_norm.values):
    X_1_ready_w2v[i] = get_embedding(text, ready_w2v, dim)
    
for i, text in enumerate(data.text_2_norm.values):
    X_2_ready_w2v[i] = get_embedding(text, ready_w2v, dim)

X_ready_w2v = np.concatenate([X_1_ready_w2v, X_2_ready_w2v], axis=1)

In [102]:
distances = np.concatenate((my_w2v_dist, ready_w2v_dist, svd_dist, nmf_dist, ft_dist), axis=1)

In [101]:
clf2_lr = LogisticRegression(C=3000, class_weight='balanced')

In [103]:
%%capture --no-stdout _
print(cross_val_score(clf_lr, distances, y, scoring="f1_micro"))

[0.41839557 0.44190871 0.45190311 0.38961938 0.40968858]


Считалось ещё дольше, но результат опять не стал лучше